In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import det, inv
from scipy.special import gamma
from math import pi, ceil
from scipy.special import erfinv
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
# from statsmodels.tsa import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from random import randint
import warnings
from sklearn.metrics import mean_squared_error
from math import sqrt

warnings.filterwarnings("ignore")

In [2]:
def preprocessing(data, method):
    ts_train = []
    ts_valid = []
    ts_test = []
    ts_param = []
    flag_n = False
    flag_diff = False
    flag_dec = False
    if method is 'normalization':
        flag_n = True
    elif method is 'differencing':
        flag_diff = True
            
    for i in range(data.shape[1]):

        temp = list(data.iloc[6:][i].dropna())
        # print(temp)
        if len(temp) > 130:
            
            cut_off_1 = ceil(len(temp)*0.7)
            cut_off_2 = ceil(len(temp)*0.9)

            temp_train = temp[:cut_off_1]
            temp_val = temp[cut_off_1:cut_off_2]
            temp_test = temp[cut_off_2:]

            if flag_n is True:
                ts_param.append([np.mean(temp_train), np.std(temp_train), np.mean(temp_val), 
                np.std(temp_val), np.mean(temp_test), np.std(temp_test)])
                
                temp_train = (temp_train - np.mean(temp_train)) / np.std(temp_train)
                ts_train.append(temp_train)
                

                temp_val = (temp_val - np.mean(temp_val)) / np.std(temp_val)
                ts_valid.append(temp_val)

                temp_test = (temp_test - np.mean(temp_test)) / np.std(temp_test)
                ts_test.append(temp_test)
                

            elif flag_diff is True:

                temp_train = pd.Series(temp_train)
                temp_train_log_diff = temp_train - temp_train.shift()
                temp_train_log_diff[0] = temp_train[0]
                temp_train_log_diff.dropna(inplace=True)
                ts_train.append(temp_train_log_diff.values)

                temp_val = pd.Series(temp_val)
                temp_val_log_diff = temp_val - temp_val.shift()
                temp_val_log_diff[0] = temp_val[0]
                temp_val_log_diff.dropna(inplace=True)
                ts_valid.append(temp_val_log_diff.values)

                temp_test = pd.Series(temp_test)
                temp_test_log_diff = temp_test - temp_test.shift()
                temp_test_log_diff[0] = temp_test[0]
                temp_test_log_diff.dropna(inplace=True)
                ts_test.append(temp_test_log_diff.values)
                
    return ts_train, ts_valid, ts_test, ts_param

# ts_train, ts_valid, ts_test, ts_param = preprocessing(df_temp, method='normalization')

In [140]:
class AutoregressiveTree:
    
    def __init__(self, p, u0=0, alpha_u=1, X=None):
        self._X = X  # Exogenous data
        
        erf_temp = np.zeros([7,1])
        for i in range(1,8):
            erf_temp[i-1] = erfinv((i/4) - 1)
        
        self._erf = erf_temp
        self._p = p
        self._alpha_W = p + 2

        self._u0 = u0
        self._alpha_u = alpha_u
        self.target = []
        self.exog = []
    #  calculate the sample mean of the data (could be a vector), maybe split into sample mean of each variable
    def sample_mean(self, data):
        # print(sum(data), len(data))
        return sum(np.asarray(data))/len(data)

    
    # calculate the scatter matrix around the mean uN
    # def scatter_matrix(self, data, uN_):
    def scatter_matrix(self, data, uN_):

        # assert data.shape[1] == p * (1 + no_exog_vars), 'Data dimensions do not match expected shape'
        # Assuming data has been preprocessed to include lags of y and X
        temp = data - uN_

        SN = 0
        for row in temp:
            row = row[:, np.newaxis]
            SN += row * row.T
        # print(len(SN),len(SN[0]))
        return SN
    # def WN_func(self, uN_, SN, W0, N):
    def WN_func(self, uN_, SN, W0, N):
        # assert SN.shape[0] == p * (1 + no_exog_vars), 'Scatter matrix dimensions do not match expected shape'
        temp = self._u0 - uN_

        # Assuming scatter matrix has been computed from preprocessed data
        temp = temp[:, np.newaxis]
        WN = W0 + SN + ((self._alpha_u * N) / (self._alpha_u + N)) * np.dot(temp, temp.T)
        return WN
    # Updates the matrix WN_d, calculates the within node covariance matrix 
    def WN_d_func(self, u0_, uN_d_, SN_d_, W0_, N_):
        temp = u0_ - uN_d_
        temp = temp[:, np.newaxis]
        WN_ = W0_ + SN_d_ + ((self._alpha_u * N_) / (self._alpha_u + N_)) * np.dot(temp, temp.T)
        return WN_
    # calculating the Maximum a posteriori arameters for the ar model
    def MAP_param(self, N, uN_, WN):
        ut = ((self._alpha_u * self._u0) + (N * uN_)) / (self._alpha_u + N)
        Wt_inv = (1 / (self._alpha_W + N - (self._p + 1))) * WN
        return ut, Wt_inv
    # calculate all of the AR parameters needed 
    def param(self, target, exog):
        # exog = np.array(exog)
        # exog2 = np.reshape(exog.transpose(1, 0, 2), (len(exog[0]), len(exog)*len(exog[0][0])))
        # data = np.hstack((target, exog2))
        self.target, self.exog = target, exog
        data = []

        for t, e in zip(target, np.asarray(exog).transpose(1, 0, 2)):
            data.append(np.concatenate([t, e.flatten()]))

        # Concatenate list1 and reshaped list2
        # print("datalen:", np.asarray(data).shape)
        N = len(data)
        uN_ = self.sample_mean(data)
        SN = self.scatter_matrix(data, uN_)
        W0 = np.identity(SN.shape[0])
        WN = self.WN_func(uN_, SN, W0, N)
        ut, Wt_inv = self.MAP_param(N, uN_, WN)
        W = inv(Wt_inv)
        var = 1 / W[-1, -1]
        Wpp = inv(Wt_inv[:-1, :-1])
        print(Wpp.shape)
        b = np.zeros([len(data[0]), 1])
        for j in range(len(b)):
            for i in range(self._p):
                b[j] += Wt_inv[-1, i] * Wpp[i, j]
        
        m = ut[-1]
        for i in range(self._p):
            m += b[i] * ut[i]
        
        return var, b, m[0]
    # scaling function using the gamma distribution
    def c_func(self, l, alpha):
        c = 1
        #   for loop goes from 1 to l
        for i in range(1, l + 1):
            c *= gamma((alpha + 1 - i) / 2)
        
        return c
    # probability density scaling function used
    def pds_func(self, N, W0, WN):
        pds = (pi**(-((self._p + 1) * N) / 2)) + \
        ((self._alpha_u / (self._alpha_u + N))**((self._p + 1) / 2)) + \
        (self.c_func(self._p + 1, self._alpha_W + N) / self.c_func(self._p + 1, self._alpha_W)) * (det(W0)**(self._alpha_W / 2))*(det(WN)**(-(self._alpha_W + N) / 2))
        return pds
    # similiar to above just now with different params
    def pd_s_func(self, u0_, N_, W0_, WN_):
        pds = (pi**(-((self._p + 1) * N_) / 2)) + \
        ((self._alpha_u / (self._alpha_u + N_))**((self._p + 1) / 2)) + \
        (self.c_func(self._p + 1, self._alpha_W - 1 + N_) / self.c_func(self._p + 1, self._alpha_W - 1)) * (det(W0_)**((self._alpha_W - 1) / 2))*(det(WN_)**(-(self._alpha_W - 1 + N_) / 2))
        return pds
    def mult_func(self, l, alpha, N):
        c = 1
        #   for loop goes from 1 to l
        for i in range(1, l + 1):
            c *= ((alpha + 1 + N - i)/(alpha + 1 - i))
        
        return c

    def pds_func2(self, N, W0, WN, u0_, N_, W0_, WN_):

        pds = (det(W0)**(self._alpha_W / 2))*det(WN)**(-(self._alpha_W + N) / 2) / (det(W0_)**((self._alpha_W - 1) / 2))*(det(WN_)**(-(self._alpha_W - 1 + N_) / 2)) * \
        self.mult_func(self._p + 1,self._alpha_W, N)                                                                                                             

        return pds

    def LeafScore(self, data):
        N = len(data)
        self.target = data
        uN_ = self.sample_mean(data)
        SN = self.scatter_matrix(data, uN_)
        W0 = np.identity(SN.shape[0])
        WN = self.WN_func(uN_, SN, W0, N)
        ut, Wt_inv = self.MAP_param(N, uN_, WN)
        data_ = []
        for x in data:
            data_.append(x[:-1])
        N_ = len(data_)
        uN_d_ = self.sample_mean(data_)
        SN_d_ = self.scatter_matrix(data_, uN_d_)
        u0_ = ut[:-1]
        W0_ = inv(inv(W0)[:-1, :-1])
        WN_d_ = self.WN_d_func(u0_, uN_d_, SN_d_, W0_, N_)

        if N > 20:
            pds2 = self.pds_func2(N, W0, WN, u0_, N_, W0_, WN_d_)
        else:
            pds = self.pds_func(N, W0, WN)
            pds_ = self.pd_s_func(u0_, N_, W0_, WN_d_)
            pds2 =  pds/pds_
        # print("frac: ", pds/pds_)
        # print("frac2: " , pds2)
        return pds2
        
    # This will spplit a dataset into two froups based on the specific features. Then splits the data points into the left or right set
    def test_split(self, index, value, dataset):
        left, right = list(), list()
        for row in dataset:
            if row[index] < value:
                left.append(row)
            else:
                right.append(row)
        return left, right
    def rest_split(self, index, value, train, exog, ex, var):
                # Initializations
        left, right = [], []
        left_e, right_e = [], []

        # Convert to DataFrame for easier operations
        train_df = pd.DataFrame(train)
        exog_dfs = [pd.DataFrame(ex) for ex in exog]

        # Split based on exogenous variable
        if ex == "y":
            # Split the specified variable in exog
            for i, row in exog_dfs[var].iterrows():
                if row[index] < value:
                    left_e.append(i)
                else:
                    right_e.append(i)

            left_e_index = pd.Index(left_e)
            right_e_index = pd.Index(right_e)

            # Extract corresponding rows from other variables in exog and from train
            left_exog = [df.loc[left_e_index] for df in exog_dfs]
            right_exog = [df.loc[right_e_index] for df in exog_dfs]

            left = train_df.loc[left_e_index]
            right = train_df.loc[right_e_index]

        # Split based on training data
        else:
            for i, row in train_df.iterrows():
                if row[index] < value:
                    left.append(i)
                else:
                    right.append(i)
            
            left_index = pd.Index(left)
            right_index = pd.Index(right)

            left = train_df.loc[left_index]
            right = train_df.loc[right_index]

            left_exog = [df.loc[left_index] for df in exog_dfs]
            right_exog = [df.loc[right_index] for df in exog_dfs]
        left = left.values.tolist()
        right = right.values.tolist()
        left_exog = [df.values.tolist() for df in left_exog]
        right_exog = [df.values.tolist() for df in right_exog]
        return left, right, left_exog, right_exog
    # itrates through the features and the values to det the best for splitting the dataset, calkculates the score for each split and choses the one with best improvement
    def get_split(self, train, train_exog):
        b_index, b_value, b_groups, var = 999, 999, None, 'y'
        b_score = self.LeafScore(train)
        avg = np.mean(train, axis=0)[:-1]
        sigma = np.std(train, axis=0)[:-1]
        split_data = train, train_exog
        for index in range(len(avg)):
            for i in range(len(self._erf)):

                value = avg[index] + sigma[index] * self._erf[i]
                groups = self.test_split(index, value, train)
                data = self.rest_split(index, value, train, train_exog,"n", 0)
                
                new_score = 1
                for group in groups:
                    if len(group) != 0:
                        new_score *= self.LeafScore(group)
            
                        if new_score > b_score:
                            b_index, b_value, b_score, b_groups, var, split_data = index, value, new_score, groups, ("y"+str(index)), data
        j = 0
        bb_score = 1
        for ex in train_exog:
            # print(j)
            avg = np.mean(ex, axis=0)[:-1]
            bb_score =  max(bb_score, self.LeafScore(ex))
            # print(avg)
            sigma = np.std(ex, axis=0)[:-1]
            for index in range(len(avg)):
                for i in range(len(self._erf)):
                    value = avg[index] + sigma[index] * self._erf[i]
                    groups = self.test_split(index, value, ex)
                    data = self.rest_split(index, value, train, train_exog,"y", j)
                    new_score = 1
                    for group in groups:
                        if len(group) != 0:
                            new_score *= self.LeafScore(group)
                
                            if new_score > b_score:
                                b_index, b_value, b_score, b_groups, var, split_data = index, value, new_score, groups, ("x"+str(j)+str(index)), data
            j += 1
        # print({'index':b_index, 'variable': var, 'value':b_value, 'groups':b_groups})
        # print(var, b_score, bb_score)
        # print(np.asarray(data).shape)
        # self.exog = exog
        return {'index':b_index, 'variable': var, 'value':b_value, 'groups':b_groups, 'split_data':split_data}
    # turns a group of points, belonging to one datagroup into a terminal node, calculates the parameters for that specific group
    def to_terminal(self, target, exog):
        outcomes = self.param(target,exog)
        return outcomes
    # this recursivelky builds the tree up. If the node should be a terminal node make terminal, else use get split to find next best split
    def split(self, node, max_depth, min_size, depth):
        left, right = node['groups']
        left_t, right_t, left_e, right_e = node['split_data']
        del(node['groups'])
        if not left or not right:
            node['left'] = node['right'] = self.to_terminal(left_t + right_t, left_e + right_e)
            return
        
        if depth >= max_depth:
            node['left'], node['right'] = self.to_terminal(left_t,left_e ), self.to_terminal(right_t, right_e)
            return
        
        if len(left) <= min_size:
            node['left'] = self.to_terminal(left_t, left_e)
        else:
            node['left'] = self.get_split(left_t, left_e)
            if node['left']['groups'] is None:
                node['left'] = self.to_terminal(left_t, left_e)
            else:
                self.split(node['left'], max_depth, min_size, depth+1)
        
        if len(right) <= min_size:
            node['right'] = self.to_terminal(right_t, right_e)
        else:
            node['right'] = self.get_split(right_t, right_e)
            if node['right']['groups'] is None:
                node['right'] = self.to_terminal(right_t, right_e)
            else:
                self.split(node['right'], max_depth, min_size, depth+1)
    # initiates the buiilding process. Finds initial split and if there are no effective splits then makes source node a terminal node
    def build_tree(self, train, train_exog, max_depth, min_size):
        train=train
        train_exog = train_exog
        root = self.get_split(train, train_exog)
        if root['groups'] is None:
            root['root'] = self.to_terminal(train,train_exog)
            root['index'] = None
            root['value'] = None
            del(root['groups'])
        else:
            # print(root['index'])
            self.split(root, max_depth, min_size, 1)
        
        return root
    # prints the tree structure 
    def print_tree(self, node, depth=0):
        if isinstance(node, dict):
            if node['value'] is None:
                print(node)
                return                                                                                                                               
            print('%s[%s < %.3f]' % ((depth*' ', (node['variable']), node['value'])))
            # print(depth)
            # print(node)
            self.print_tree(node['left'], depth+1)
            self.print_tree(node['right'], depth+1)
    
        else:
            print('%s[%s]' % ((depth*' ', node)))
    # follows the tree strarting from root node until a terminal node is found
    def predict(self, node, row):
        # Expecting the input row to include lags of both y and X
        if 'root' in node:
            return node['root']
        if row[node['index']] < node['value']:
            if isinstance(node['left'], dict):
                return self.predict(node['left'], row)
            else:
                return node['left']
        else:
            if isinstance(node['right'], dict):
                return self.predict(node['right'], row)
            else:
                return node['right']


In [38]:
def generate_synthetic_data_df_multivar(n_samples=200, n_exog_vars=4):
    """
    Generates a synthetic dataset with a clear relationship between the target and multiple exogenous variables.
    Returns the data as pandas DataFrames.
    """
    exog_data = {}
    
    for i in range(n_exog_vars):
        # Exogenous variable
        exog_data[i] = np.linspace(0, 10*(i+1), n_samples) + np.random.normal(0, 1, n_samples)
    
    # Create a DataFrame for exogenous variables
    exog_df = pd.DataFrame(exog_data)
    
    # Target variable influenced by the sum of exogenous variables with some noise
    target = exog_df.sum(axis=1) + 5 * np.sin(exog_df[0]) + np.random.normal(0, 2, n_samples)
    target_df = pd.DataFrame(target, columns=[0])
    
    return target_df, exog_df

def plot_data_distributions_grid(target_df, exog_df):
    """
    Plots the distributions of the target and multiple exogenous variables in a grid layout.
    """
    n_exog_vars = exog_df.shape[1]
    
    fig, axes = plt.subplots(nrows=1, ncols=n_exog_vars+1, figsize=(8, 3))

    # Plotting target variable distribution
    target_df[0].hist(ax=axes[0], bins=30, color='blue', alpha=0.3)
    axes[0].set_title("Target Variable Distribution")
    axes[0].set_xlabel("Value")
    axes[0].set_ylabel("Frequency")

    # Plotting exogenous variable distributions
    for i, col in enumerate(exog_df.columns):
        exog_df[col].hist(ax=axes[i+1], bins=30, color='green', alpha=0.3)
        axes[i+1].set_title(f"{col} Distribution")
        axes[i+1].set_xlabel("Value")

    plt.tight_layout()
    plt.show()


# Generate synthetic data with multiple exogenous variables as DataFrames
# synthetic_target_df_multivar, synthetic_exog_df_multivar = generate_synthetic_data_df_multivar()

# Plot distributions for the synthetic data
synthetic_target, synthetic_exog = generate_synthetic_data_df_multivar()
print(len(ts_train),len(ts_train[0]))#,len(ts_train[0][0]))

5 136


In [141]:
p = 3
df_temp2 = pd.concat([synthetic_target,synthetic_exog], axis=1)
df_temp2.columns = range(df_temp2.shape[1])

ts_train, ts_valid, ts_test, ts_param = preprocessing(df_temp2, method='normalization')

train = []
for s in ts_train:
    temp = []
    for i in range(len(s) - (p + 1)):
        temp.append(s[i:i + p + 1])


    train.append(temp)
d = train[0]
# print(len(d), len(d[0]))
d_exog = train[1:]
ART = AutoregressiveTree(3)
tree = ART.build_tree(d, d_exog, 10, 3)

ART.print_tree(tree)


(19, 19)


IndexError: index 19 is out of bounds for axis 1 with size 19

In [133]:
target, exog = ART.target, ART.exog
print(len(target),len(target[0]), len(exog), len(exog[0]), len(exog[0][0]))

12 4 4 12 4


In [137]:
data = []

for t, e in zip(target, np.asarray(exog).transpose(1, 0, 2)):
    data.append(np.concatenate([t, e.flatten()]))

print(len(data),len(data[0]))
print(data)

12 20
[array([1.18391606, 1.66815844, 1.30268093, 1.05600429, 1.14278682,
       1.8782003 , 1.2145323 , 0.58739017, 1.30367948, 0.9057819 ,
       1.65597517, 1.03254549, 0.75849789, 0.94634346, 1.14918061,
       1.37566994, 0.93499502, 1.29376193, 1.24232053, 1.26033115]), array([1.33878627, 1.2810999 , 1.41948887, 1.64159771, 0.97557114,
       1.02591157, 1.03288664, 1.48087115, 1.38256264, 1.11677296,
       1.14271338, 1.28082294, 1.16203454, 1.47240565, 1.28009631,
       1.31550214, 1.29423833, 1.26787635, 1.46214552, 1.2125783 ]), array([1.2810999 , 1.41948887, 1.64159771, 1.77973009, 1.02591157,
       1.03288664, 1.48087115, 1.39210565, 1.11677296, 1.14271338,
       1.28082294, 1.44711657, 1.47240565, 1.28009631, 1.31550214,
       1.53991943, 1.26787635, 1.46214552, 1.2125783 , 1.4083515 ]), array([1.41948887, 1.64159771, 1.77973009, 1.56173255, 1.03288664,
       1.48087115, 1.39210565, 1.64460022, 1.14271338, 1.28082294,
       1.44711657, 1.13773076, 1.28009631, 1.3155

data = []

for d1, d2 in zip(target, exog):
            data_sub = d1 + [item for sublist in d2 for item in sublist]
            data.append(data_sub)


print(len(data),len(data[0]))

the input dimensions are for target(12,4) and exog (4,12,4). I need the merged dataframe to be of dimension (12, 20) with first the target and then the exog variables 

In [135]:
np.asarray(data)

array([[1.18391606, 1.66815844, 1.30268093, 1.05600429, 1.14278682,
        1.8782003 , 1.2145323 , 0.58739017, 0.97557114, 1.02591157,
        1.03288664, 1.48087115, 1.02591157, 1.03288664, 1.48087115,
        1.39210565, 1.03288664, 1.48087115, 1.39210565, 1.64460022,
        1.48087115, 1.39210565, 1.64460022, 0.9844004 , 1.39210565,
        1.64460022, 0.9844004 , 1.40497144, 1.64460022, 0.9844004 ,
        1.40497144, 2.43461393, 0.9844004 , 1.40497144, 2.43461393,
        0.64413495, 1.27964475, 1.30587014, 1.27356531, 1.56370247,
        1.30587014, 1.27356531, 1.56370247, 1.09238956, 1.27356531,
        1.56370247, 1.09238956, 1.76051784, 1.56370247, 1.09238956,
        1.76051784, 0.53044597],
       [1.33878627, 1.2810999 , 1.41948887, 1.64159771, 1.30367948,
        0.9057819 , 1.65597517, 1.03254549, 1.38256264, 1.11677296,
        1.14271338, 1.28082294, 1.11677296, 1.14271338, 1.28082294,
        1.44711657, 1.14271338, 1.28082294, 1.44711657, 1.13773076,
        1.28082

In [96]:
def rest_split(index, value, train, exog, ex, var):
                # Initializations
        left, right = [], []
        left_e, right_e = [], []

        # Convert to DataFrame for easier operations
        train_df = pd.DataFrame(train)
        exog_dfs = [pd.DataFrame(ex) for ex in exog]

        # Split based on exogenous variable
        if ex == "y":
            # Split the specified variable in exog
            for i, row in exog_dfs[var].iterrows():
                if row[index] < value:
                    left_e.append(i)
                else:
                    right_e.append(i)

            left_e_index = pd.Index(left_e)
            right_e_index = pd.Index(right_e)

            # Extract corresponding rows from other variables in exog and from train
            left_exog = [df.loc[left_e_index] for df in exog_dfs]
            right_exog = [df.loc[right_e_index] for df in exog_dfs]

            left = train_df.loc[left_e_index]
            right = train_df.loc[right_e_index]

         # Split based on training data
        else:
            for i, row in train_df.iterrows():
                if row[index] < value:
                    left.append(i)
                else:
                    right.append(i)
            
            left_index = pd.Index(left)
            right_index = pd.Index(right)

            left = train_df.loc[left_index]
            right = train_df.loc[right_index]

            left_exog = [df.loc[left_index] for df in exog_dfs]
            right_exog = [df.loc[right_index] for df in exog_dfs]
        left = left.values.tolist()
        right = right.values.tolist()
        left_exog = [df.values.tolist() for df in left_exog]
        right_exog = [df.values.tolist() for df in right_exog]

        return left, right, left_exog, right_exog

left, right, left_exog, right_exog = rest_split(0, -1, d, d_exog, "b", 0)

In [22]:
def time_series_pred(data, time_series_index, pred_method, max_p, preprocessing_method):
    ts_train, ts_valid, ts_test, ts_param = preprocessing(data, method=preprocessing_method)
    p_set = [i for i in range(1,max_p+1)]
    mse_our_model = []
    mse_arima = []
    valid_prediction_list = []
    valid_prediction_list_cumsum = []
    #   Example
    idx = time_series_index
    d_val = ts_valid[0][1:]

    d_test = ts_test[0]
    tree_list = []
    for p in p_set:
        train = []
        for s in ts_train:
            temp = []
            for i in range(len(s) - (p + 1)):
                temp.append(s[i:i + p + 1])

            train.append(temp)
        # print(len(train),len(train[0]))
        d = train[0]
        d_exog = train[1:]
        # print(len(d),len(d[0]))
        # print(len(d_exog),len(d_exog[0]))
        ART = AutoregressiveTree(p)
        tree = ART.build_tree(d, d_exog, 100, 5)
        tree_list.append(tree)
        valid_prediction = []
        valid_window = d[-1][1:]
        moving_val = ts_train[0]
        if pred_method is 'ART':
            for i in range(len(d_val)):
                parameters = ART.predict(tree, valid_window)
                prediction_temp = np.dot(valid_window[:,np.newaxis].T,parameters[1]) + parameters[2]
                valid_prediction.append(prediction_temp[0][0])
                valid_window = np.append(valid_window, d_val[i])
                valid_window = valid_window[1:]
            valid_prediction_list.append(valid_prediction)
            valid_prediction = pd.Series(valid_prediction, copy=True)
            if preprocessing_method is 'differencing':
                train_s = pd.Series(ts_train[idx], copy=True).cumsum()
                last_value_train= pd.Series.tolist(train_s)[-1]
                valid_prediction_temp = [0]*(len(valid_prediction)+1)
                valid_prediction_temp[1:] = valid_prediction
                valid_prediction_temp[0] = last_value_train
                valid_prediction_temp = pd.Series(valid_prediction_temp, copy=True)
                valid_prediction_cumsum = valid_prediction_temp.cumsum()
                valid_prediction_list_cumsum.append(valid_prediction_cumsum)
            elif preprocessing_method is 'normalization':
                d_val_mean = ts_param[idx][2]
                d_val_std = ts_param[idx][3]
                valid_prediction_denorm= (valid_prediction * d_val_std) + d_val_mean
                valid_prediction_list_cumsum.append(valid_prediction_denorm)
        elif pred_method is 'AR':
            for i in range(len(d_val)):
                #           alid_window = np.append(valid_window, prediction)
                moving_val = moving_val[~np.isnan(moving_val)]
                model = ARIMA(moving_val, order=(p,1,0))
                model_fit = model.fit()
                prediction_temp = model_fit.forecast()
                valid_prediction.append(prediction_temp[0])
                moving_val = np.append(moving_val, d_val[i])
                d_val = pd.Series(d_val, copy=True)
            valid_prediction = pd.Series(valid_prediction, copy=True)
            valid_prediction_list.append(valid_prediction)
            if preprocessing_method is 'differencing':
                train_s = pd.Series(ts_train[idx], copy=True).cumsum()
                last_value_train= pd.Series.tolist(train_s)[-1]
                valid_prediction_temp = [0]*(len(valid_prediction)+1)
                valid_prediction_temp[1:] = valid_prediction
                valid_prediction_temp[0] = last_value_train
                valid_prediction_temp = pd.Series(valid_prediction_temp, copy=True)
                valid_prediction_cumsum = valid_prediction_temp.cumsum()
                valid_prediction_list_cumsum.append(valid_prediction_cumsum)
            elif preprocessing_method is 'normalization':
                d_val_mean = ts_param[idx][2]
                d_val_std = ts_param[idx][3]
                valid_prediction_denorm= (valid_prediction * d_val_std) + d_val_mean
                valid_prediction_list_cumsum.append(valid_prediction_denorm)

    if preprocessing_method is 'differencing':
        d_val_cumsum = ts_valid[idx].cumsum()
    elif preprocessing_method is 'normalization':
        d_val_mean = ts_param[idx][2]
        d_val_std = ts_param[idx][3]
        d_val_cumsum = (d_val * d_val_std) + d_val_mean
    else:
        d_val_cumsum = d_val
        
        
    return d_val, d_val_cumsum, valid_prediction_list, valid_prediction_list_cumsum, tree_list

# Ground_truth_diff, Ground_truth, valid_prediction_diff_ART_list, valid_prediction_ART_list, tree_list = time_series_pred(data=df_temp, 
                                # time_series_index=0, pred_method='ART', max_p=5, preprocessing_method='normalization')

In [36]:
max_p=3
hit_rate_ART_all_samples = []
hit_rate_AR_all_samples = []
rmse_ART_all_samples = []
rmse_AR_all_samples = []
time_series_index_list = [350] #, 786, 396, 276, 614, 377, 101, 124, 62, 35]
for i in range(0,1):
    time_series_index = 0
    print('Processing... time series number: ',time_series_index)
    time_series_index_list.append(time_series_index)
    Ground_truth_diff, Ground_truth, valid_prediction_diff_ART_list, valid_prediction_ART_list, tree = time_series_pred(data=df_temp, 
                                    time_series_index=time_series_index, pred_method='ART', max_p=max_p, preprocessing_method='normalization')
    _, _, valid_prediction_diff_AR_list, valid_prediction_AR_list, _ = time_series_pred(data=df_temp, 
                                    time_series_index=time_series_index, pred_method='AR', max_p=max_p, preprocessing_method='normalization')

    hit_rate_ART_list = []
    hit_rate_AR_list = []
    rmse_ART_list = []
    rmse_AR_list = []
    for i in range(max_p):
        hit_rate_ART_list.append(hit_rate(Ground_truth, valid_prediction_ART_list[i]))
        hit_rate_AR_list.append(hit_rate(Ground_truth, valid_prediction_AR_list[i]))
        rmse_ART_list.append(sqrt(mean_squared_error(Ground_truth, valid_prediction_ART_list[i])))
        rmse_AR_list.append(sqrt(mean_squared_error(Ground_truth, valid_prediction_AR_list[i])))
    hit_rate_ART_all_samples.append(hit_rate_ART_list)
    hit_rate_AR_all_samples.append(hit_rate_AR_list)
    rmse_ART_all_samples.append(rmse_ART_list)
    rmse_AR_all_samples.append(rmse_AR_list)

Processing... time series number:  0


In [4]:
def hit_rate(ts_true, ts_pred):
    diff_true = np.diff(ts_true)
    diff_pred = np.diff(ts_pred)
    return np.sum(np.sign(diff_true) == np.sign(diff_pred)) / len(diff_true)

In [62]:
# ------ Functions:
def read_data_of_a_stock(dataset, name_of_stock):
    # taking the column of dataset having the name of stocks:
    names_of_stocks = dataset[:, 1]
    # find the indices of a specific stock:
    stock_index = np.where(names_of_stocks == name_of_stock)
    dataset_selected_stock = dataset[stock_index[0], :]
    return dataset_selected_stock

def extract_features_from_dataset(dataset):
    dataset_only_features = dataset[:, 2:]
    return dataset_only_features

# ------ Settings:
path_dataset = 'prices.csv'
path_save = './Data/saved_files/'    # GE, GOOG, ORCL and JNJ have no fundamental features, GM has not enough technical samples
# 'AAPL', 'AIG', 'AMZN', 'BA', 'CAT', 'COF', 'EBAY', 'F', 'FDX', 'GE', 'GM', 'GOOG', 'HD', 'IBM', 'JNJ', 'JPM', 'KO', \
# 'MSFT', 'NFLX', 'NKE', 'ORCL', 'PEP', 'T', 'WMT', 'XOM', 'XRX'
stocks_to_process = ['AMZN', 'GE', 'CAT','GM', 'IBM', 'AAPL', 'AMZN', 'NFLX', 'F']
# stocks_to_process = ['AAPL', 'AIG', 'AMZN','BA']
number_of_stocks = len(stocks_to_process)
# print(number_of_stocks)
# ------ loading dataset:
data_df = pd.read_csv(path_dataset, delimiter=',')
data = data_df.values  # converting pandas data frame to numpy array
# ------ Preprocessing & time series forcasting:
df_temp = pd.DataFrame()
print('Status of code: Preprocessing & time series forcasting.....')
for stock_index in range(len(stocks_to_process)):
    name_of_stock = stocks_to_process[stock_index]

    data_a_stock = read_data_of_a_stock(dataset=data, name_of_stock=name_of_stock)

    # ------ pre-processing of dataset:
    data_a_stock = extract_features_from_dataset(dataset=data_a_stock)
    total_number_of_days = data_a_stock.shape[0]
    closing = data_a_stock[:, 1].tolist()
    # opening = data_a_stock[:, 0].tolist()
    # lowest = data_a_stock[:, 2].tolist()
    # highest = data_a_stock[:, 3].tolist()
    df_temp[stock_index] = pd.DataFrame(data=closing)

Status of code: Preprocessing & time series forcasting.....


In [6]:
df_temp

,0,1,2,3,4,5,6,7,8
0,133.899994,15.450000,58.549999,34.189999,132.449997,214.009998,133.899994,53.479999,10.28
1,134.690002,15.530000,59.250000,34.259998,130.850006,214.379993,134.690002,51.510001,10.96
2,132.250000,15.450000,59.430000,34.080002,130.000000,210.969995,132.250000,53.319999,11.37
3,130.000000,16.250000,59.669998,33.250000,129.550003,210.580000,130.000000,52.400001,11.66
4,133.520004,16.600000,60.340000,33.480000,130.850006,211.980005,133.520004,53.300002,11.69
...,...,...,...,...,...,...,...,...,...
1757,760.590027,31.879999,94.320000,NaN,166.710007,116.519997,760.590027,125.589996,12.46
1758,771.400024,31.900000,94.550003,NaN,167.139999,117.260002,771.400024,128.350006,12.39
1759,772.130005,31.700001,93.150002,NaN,166.190002,116.760002,772.130005,125.889999,12.25
1760,765.150024,31.709999,93.290001,NaN,166.600006,116.730003,765.150024,125.330002,12.23


In [31]:
Ground_truth_diff, Ground_truth, valid_prediction_diff_ART_list, valid_prediction_ART_list, tree_list = time_series_pred(data=df_temp, 
                                time_series_index=0, pred_method='ART', max_p=5, preprocessing_method='normalization')

In [32]:
ART = AutoregressiveTree(5)
ART.print_tree(tree_list[0])

{'index': None, 'variable': 'y', 'value': None, 'root': (0.006004329905967443, array([[0.99680884]]), -0.0020688227333991766)}


0
1
2


Hit rate for ART model( p: 3 )after prep 0.4871060171919771


In [39]:
max_p=3
hit_rate_ART_list = []
hit_rate_AR_list = []
for i in range(max_p):
    print(i)
    hit_rate_ART_list.append(hit_rate(Ground_truth, valid_prediction_ART_list[i]))
    
max_hit_rate_index, max_hit_rate_value = max(enumerate(hit_rate_ART_list)) 
a = list(Ground_truth)
b = list(valid_prediction_ART_list[max_hit_rate_index])
plt.plot(a[1:], label='Original time series')
plt.plot(b[1:], label='Autoregressive Tree')
plt.xlabel('time')
plt.ylabel('price')
i=max_hit_rate_index+1
plt.title('prediction based on last %i values in time series' %i)
plt.legend()
plt.show()
print('Hit rate for ART model(','p:',max_hit_rate_index+1,')after prep', max_hit_rate_value)
plt.plot(range(1, max_p+1), hit_rate_ART_list, label='Autoregressive Tree')
plt.xlabel('p')
plt.ylabel('Hit rate')
plt.title('Hit rate for ART model and AR model')
plt.legend()
plt.show()